In [2]:
import pandas as pd

## Load 

In [3]:
orders_dirty = pd.read_csv('../data/raw/orders_dirty_v1.csv')

In [4]:
orders_dirty

,order_id,customer_ref,order_ts,ship_ts,status,items_json,subtotal,tax,shipping_fee,discount_code,discount_amount,payment_method,card_last4,currency,ship_country,ship_postal_code,ip_address,user_agent
0,A-1001,CUST-001,2025-04-12 14:22:10,2025-04-13 09:10:00,delivered,"[{""sku"":""SKU-001"",""qty"":2,""unit_price"":""19.99""...",49.48,4.95,0,SPRING10,-5.0,card,1234,EUR,France,75001,192.168.1.10,Mozilla/5.0
1,A-1002,cust-002,12/04/2025 15:00,NaN,shipped,"[{""sku"":""SKU-003"",""qty"":""1"",""unit_price"":""$12....",12.40,1.24,5.00,NaN,NaN,paypal,NaN,USD,US,02139,10.0.0.5,curl/8.0
2,A-1003,CUST-003,2025-02-30 10:00:00,NaN,paid,not_json,19.99,0,0,WELCOME,0.0,card,9999,EUR,DE,10115,256.256.1.1,Mozilla/5.0
3,A-1004,CUST-004,2025-03-01T09:00:00Z,2025-03-02T10:00:00Z,CANCELLED,"[{""sku"":""SKU-001"",""qty"":1,""unit_price"":19.99}]",19.99,"3,80",FREE,VIP,5.0,card,12A4,EUR,Germany,D-10115,203.0.113.5,PostmanRuntime/7.32.2
4,A-1005,NaN,2025-03-05 12:00:00,NaN,paid,"[{""sku"":""SKU-010"",""qty"":0,""unit_price"":""15""}]",15,3,5,WELCOME10,-3.0,bank_transfer,NaN,EUR,FR,7500A,198.51.100.7,Mozilla/5.0
5,A-1006,CUST-006,2026-12-01 08:00:00,NaN,paid,"[{""sku"":""SKU-002"",""qty"":1,""unit_price"":""9.50""}]",9.50,0.95,2.00,NaN,NaN,card,0000,EUR,FR,75002,203.0.113.99,Mozilla/5.0
6,A-1007,CUST-007,2025-03-10 18:30:00,2025-03-11 07:00:00,delivered,"[{""sku"":""SKU-005"",""qty"":2,""unit_price"":""12,00""}]","24,00","2,40","2,99",SPRING10,2.4,paypal,NaN,EUR,FRA,13001,192.0.2.1,Mozilla/5.0
7,A-1008,CUST-008,2025-03-10 18:30:00,2025-03-10 17:00:00,delivered,"[{""sku"":""SKU-005"",""qty"":1,""unit_price"":""12.00""}]",12.00,1.20,2.99,NaN,NaN,card,4321,EUR,FR,13001,192.0.2.1,Mozilla/5.0
8,A-1009,CUST-009,2025-01-10 08:00:00,NaN,refund,"[{""sku"":""SKU-007"",""qty"":1,""unit_price"":""0""}]",0,0,0,NaN,NaN,card,1111,EUR,IT,00100,203.0.113.12,Mozilla/5.0
9,A-1010,CUST-010,2025-01-10 08:00:00,NaN,paid,"[{""sku"":""SKU-008"",""qty"":1,""unit_price"":""1e2""}]",100,20,10,NEW,10.0,card,2222,EUR,UK,SW1A1AA,203.0.113.12,Mozilla/5.0


- Problèmes majeurs

Dates invalides (2025-02-30)

ship_ts < order_ts

items_json non valide (not_json)

IP invalide (256.256.1.1)

- Incohérences de format

Formats de date différents

status casse différente (CANCELLED)

Montants avec virgule (24,00, 3,80), subtotal et shipping_fee

shipping_fee = "FREE"

unit_price avec $, virgule, ou 1e2

card_last4 invalide (12A4)

Pays hétérogènes (FR, France, FRA…)

Codes postaux mal formatés

## Analyse and describe

In [5]:
orders_dirty.info()

<class 'pandas.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   order_id          11 non-null     str    
 1   customer_ref      10 non-null     str    
 2   order_ts          11 non-null     str    
 3   ship_ts           4 non-null      str    
 4   status            11 non-null     str    
 5   items_json        11 non-null     str    
 6   subtotal          11 non-null     str    
 7   tax               11 non-null     str    
 8   shipping_fee      11 non-null     str    
 9   discount_code     7 non-null      str    
 10  discount_amount   7 non-null      float64
 11  payment_method    11 non-null     str    
 12  card_last4        8 non-null      str    
 13  currency          11 non-null     str    
 14  ship_country      11 non-null     str    
 15  ship_postal_code  11 non-null     str    
 16  ip_address        11 non-null     str    
 17  user_agent

In [6]:
orders_dirty.describe()

,discount_amount
count,7.000000
mean,2.771429
std,5.930069
min,-5.000000
25%,-1.500000
50%,2.400000
75%,7.500000
max,10.000000


In [12]:
pd.read_csv('../data/processed/orders_clean_v1.csv')

,order_id,customer_ref,order_ts,ship_ts,status,items_json,subtotal,tax,shipping_fee,discount_code,discount_amount,payment_method,card_last4,currency,ship_country,ship_postal_code,ip_address,user_agent
0,A-1007,CUST-007,2025-03-10 18:30:00+00:00,2025-03-11 07:00:00+00:00,delivered,"[{'sku': 'SKU-005', 'qty': 2, 'unit_price': 12...",24.0,2.4,2.99,SPRING10,2.4,paypal,0,EUR,FR,13001,192.0.2.1,Mozilla/5.0
1,A-1009,CUST-009,2025-01-10 08:00:00+00:00,NaN,refund,"[{'sku': 'SKU-007', 'qty': 1, 'unit_price': 0.0}]",0.0,0.0,0.00,NaN,0.0,card,1111,EUR,IT,00100,203.0.113.12,Mozilla/5.0
2,A-1010,CUST-010,2025-01-10 08:00:00+00:00,NaN,paid,"[{'sku': 'SKU-008', 'qty': 1, 'unit_price': 10...",100.0,20.0,10.00,NEW,10.0,card,2222,EUR,GB,SW1A1AA,203.0.113.12,Mozilla/5.0
3,A-1010,CUST-010,2025-01-10 08:00:00+00:00,NaN,paid,"[{'sku': 'SKU-008', 'qty': 1, 'unit_price': 10...",100.0,20.0,10.00,NEW,10.0,card,2222,EUR,GB,SW1A 1AA,203.0.113.12,Mozilla/5.0
